In [2]:
import keras
from keras.layers import Input, Activation, Dense, Conv2D, MaxPooling2D,Flatten,Lambda
from keras.models import Model
import keras.backend as K
from keras.optimizers import SGD
from keras.callbacks import LearningRateScheduler

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
digit_input = Input(shape=(105,105,1))
x = Conv2D(64, (10, 10),activation='relu',input_shape=(105,105,1))(digit_input)
x = MaxPooling2D()(x)
x = Conv2D(128, (7, 7),activation='relu')(x)
x = MaxPooling2D()(x)
x = Conv2D(128, (4, 4),activation='relu')(x)
x = MaxPooling2D()(x)
x = Conv2D(256, (4, 4),activation='relu')(x)
x = Flatten()(x)
out = Dense(4096,activation='sigmoid')(x)
vision_model = Model(digit_input,out)

letter1 = Input(shape=(105,105,1))
letter2 = Input(shape=(105,105,1))

out_1 = vision_model(letter1)
out_2 = vision_model(letter2)

def m_dist(A,B):
    return K.sum(K.abs(A-B),axis=1,keepdims=True)

merged_vector = Lambda(lambda x:m_dist(x[0],x[1]), output_shape=lambda inp_shp:(inp_shp[0][0],1))([out_1,out_2])

#out_fin = Dense(4096,activation="sigmoid")(merged_vector)
final_output = Dense(1,activation="sigmoid")(merged_vector)

final_model = keras.models.Model(inputs=[letter1, letter2], outputs=final_output)

In [4]:
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.lr = []
        
    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.lr.append(step_decay(len(self.losses)))
        print('lr:', step_decay(len(self.losses)))
def step_decay(epoch):
    initial_lrate = 1e-4
    lrate = initial_lrate * math.pow(0.99,epoch)
    return lrate
loss_history = LossHistory()
lrate = LearningRateScheduler(step_decay)
callbacks_list = [loss_history, lrate]

optimizer = SGD(lr=1e-4,momentum=0.5,decay=0,nesterov=False)

In [5]:
final_model.compile(optimizer=optimizer,
                   loss='binary_crossentropy',
                   metric=['binary_accuracy'],)

In [6]:
final_model.layers[2].summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 105, 105, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 96, 96, 64)        6464      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 48, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 42, 42, 128)       401536    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 21, 21, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 18, 18, 128)       262272    
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 9, 9, 128)         0         
__________

In [ ]:
model.fit(X_train, y_train, 
        validation_data=(X_test, y_test), 
        epochs=epochs, 
        batch_size=batch_size, 
        callbacks=callbacks_list)